<a href="https://colab.research.google.com/github/sledhead/simple_python/blob/main/webpage_scrapv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
#Will display upload box to user to allow a file to upload and encrypted.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xmltodict

In [3]:
import xmltodict

Section below will gather xml file for each year of the senate vote....
2017 to current

In [4]:
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_2.xml

--2024-05-17 07:24:42--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:ec00:108c::1e3, 2600:1408:ec00:108e::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_1.xml’

vote_menu_115_1.xml     [ <=>                ] 149.40K  --.-KB/s    in 0.005s  

2024-05-17 07:24:42 (29.3 MB/s) - ‘vote_menu_115_1.xml’ saved [152990]

--2024-05-17 07:24:42--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:ec00:108c::1e3, 2600:1408:ec00:108e::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_2.xml’

vote_menu_115_2.xml     [ <=>          

Next Section below will open xml and then use this file to build the list of file links for gather senator votes

In [8]:

full_http_link_file_path = []
vote_menu_file_lst = ['vote_menu_115_1.xml','vote_menu_115_2.xml','vote_menu_116_1.xml','vote_menu_116_2.xml','vote_menu_117_1.xml','vote_menu_117_2.xml','vote_menu_118_1.xml','vote_menu_118_2.xml']

for each_year_call_lst in vote_menu_file_lst:

  xml_file = open(each_year_call_lst, 'r')
  xml_data_str = xml_file.read()

  xml_to_dict_obj = xmltodict.parse(xml_data_str)

  congress_num = xml_to_dict_obj['vote_summary']['congress']
  congress_sec_num = xml_to_dict_obj['vote_summary']['session']

  congress_lst_vote = xml_to_dict_obj['vote_summary']['votes']['vote']



  base_link_path = 'https://www.senate.gov/legislative/LIS/roll_call_votes/'
  full_base_link_path = base_link_path + 'vote' + str(congress_num) + str(congress_sec_num) + '/vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_'
  print(full_base_link_path)

  for each_vote_section in congress_lst_vote:
    single_vote_num = each_vote_section['vote_number']
    #build full link path for getting xml data....
    #https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_00420.htm

    single_file_link_path = full_base_link_path + str(single_vote_num) + '.xml'
    full_http_link_file_path.append(single_file_link_path)


https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1152/vote_115_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1161/vote_116_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1162/vote_116_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1181/vote_118_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1182/vote_118_2_


# Test Section


In [9]:
print(len(full_http_link_file_path))

2791


In [10]:
for single_file_path in full_http_link_file_path:
  print(single_file_path)



https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00325.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00324.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00323.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00322.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00321.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00320.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00319.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00318.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00317.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00316.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00315.xml
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/v

# Build Senator Dict

Section below will loop through list of xml files and build a dict of all the votes within the time period.  Just selected senators will be recorded.

# Test Section

In [ ]:
#will use this section to test the above code.